# Assignment 7: Grouping and more joins

In [1]:
%load_ext sql

In [2]:
%sql postgresql://appdev@data:5432/appdev

'Connected: appdev@appdev'

### 1. Using a single join and a grouping, write a query that allows you to find the forename, surname, driverid   and the total number of wins for each driver in the f1db schema.

In [18]:
%%sql 
SELECT driverid as driver_id,
       forename as first_name,
       surname as last_name,
       count(results.position) as wins
FROM drivers
JOIN results USING (driverid)
WHERE results.position = 1
GROUP BY (drivers.driverid)
ORDER BY wins DESC
LIMIT 10

10 rows affected.


driver_id,first_name,last_name,wins
30,Michael,Schumacher,91
1,Lewis,Hamilton,60
117,Alain,Prost,51
20,Sebastian,Vettel,46
102,Ayrton,Senna,41
4,Fernando,Alonso,32
95,Nigel,Mansell,31
328,Jackie,Stewart,27
182,Niki,Lauda,25
373,Jim,Clark,25


As you can see in the above query, we have found the 10 best drivers in the f1db schema.
It shows the driver_id, first_name, last_name and the total number of wins each driver has, where the best one is Michael Schumacher.

### 2. Write a query that joins the tables results, constructors and drivers to show the amount of times a driver has driven a car from a constructor. In other words how many times one driver (for instance Schumacher) has driven a constructor (for instance Mercedes). Your table should have three fields: drivers.driverref, constructors.name and count(*) (showing the number of times the driver has driven the constructor).

#### A. Hint: You can group on more than one value


In [20]:
%%sql
SELECT driverref as driver,
       constructors.name as constructor,
       count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
GROUP BY (drivers.driverref, constructors.name)
ORDER BY time DESC
LIMIT 10

10 rows affected.


driver,constructor,time
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136
hakkinen,McLaren,133
laffite,Ligier,132
webber,Red Bull,129
raikkonen,Ferrari,126
vettel,Red Bull,113


Here we show the total amount of time a driver has driven a car from a constructor with the 10 most drivers who's used most time.
Again it show Michael Schumacher on the top.

### 3. Now we have found the most driven vehicles even for those who didn't finish the race. Extend the query from 2 by removing all tuples from the result where the status is not "Finished".

In [33]:
%%sql 
SELECT driverref as driver,
       constructors.name as constructor,
       count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY (drivers.driverref, constructors.name)
ORDER BY time DESC
LIMIT 10

10 rows affected.


driver,constructor,time
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97
raikkonen,Ferrari,92
button,McLaren,87
alonso,Ferrari,87
coulthard,McLaren,85
hamilton,Mercedes,85


So to find the results of the finished vehicles we search for the drivers with a results.statusid = 1.

### 4. Create a third (and new) query that finds the amount of milliseconds spent in pitstop (see the pitstops relation) by each unique combinations of driverid and raceid.

#### A. Hint: use the sum aggregate function to find the total amount of milliseconds
#### B. Hint: i recommend making this into a view


In [48]:
%%sql
DROP VIEW IF EXISTS pitstop_time;
CREATE VIEW pitstop_time AS
SELECT driverid as driver_id,
       races.raceid as race_id,
       driverref as driver,
       count(pitstops.driverid) as pit_time,
       sum(pitstops.milliseconds) as milliseconds
FROM drivers
JOIN pitstops USING (driverid)
JOIN races USING (raceid)
GROUP BY (drivers.driverid, races.raceid)
ORDER BY milliseconds DESC

Done.
Done.


[]

In [49]:
%sql SELECT * FROM pitstop_time limit 10

10 rows affected.


driver_id,race_id,driver,pit_time,milliseconds
817,967,ricciardo,5,3572090
18,967,button,5,3569001
830,967,max_verstappen,5,3563738
13,967,massa,4,3554774
826,967,kvyat,4,3546767
825,967,kevin_magnussen,4,3542704
822,967,bottas,4,3542270
821,967,gutierrez,4,3538573
20,967,vettel,3,3524541
836,967,wehrlein,3,3518952


Here we create a view which shows the amount of milliseconds used in a pitstop using the driver_id and the race_id.

### 5. Time to put everything together. Use the query for 4 as a join subquery in query 3 (see slides on "a query within a query") to find the total amount of pitstop time, for each result in query 3

In [50]:
%%sql
DROP VIEW IF EXISTS finished_vehicles;
CREATE VIEW finished_vehicles AS
SELECT driverref as driver,
       constructors.name as constructor,
       count(*) as time
FROM drivers
JOIN results USING (driverid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY (drivers.driverref, constructors.name)
ORDER BY time DESC;

Done.
Done.


[]

In [62]:
%%sql
SELECT fv.driver,
       fv.constructor,
       sum(pt.milliseconds) as time
FROM finished_vehicles as fv
INNER JOIN (SELECT
              driver_id,
              race_id,
              driver,
              pitstop_time.milliseconds,
              results.constructorid,
              constructors.name
          FROM pitstop_time
          JOIN results ON results.driverid = pitstop_time.driver_id
          AND results.raceid = pitstop_time.race_id
          JOIN constructors ON constructors.constructorid = results.constructorid) as pt
ON pt.driver = fv.driver AND pt.name = fv.constructor
GROUP BY (fv.driver, fv.constructor)
ORDER BY time DESC
LIMIT 10;

10 rows affected.


driver,constructor,time
hamilton,Mercedes,12428578
rosberg,Mercedes,12390412
perez,Force India,11876732
massa,Williams,11874456
ricciardo,Red Bull,11816929
button,McLaren,11800713
bottas,Williams,10665109
hulkenberg,Force India,10198930
vettel,Ferrari,9248461
nasr,Sauber,8513533


Here an INNER JOIN is used to get the total amount of pitstop time.

### 6. As a last thing we want to find the driver with the least pistop time. However we want to control for the amount of races (see explanation below), so we need to include one last column: the average pitstop time per race. Your table should be sorted based on the ascending values of that average pitstop time.

#### A. Explanation: If we just count the pitstop time, we only get the total time he spend in a workshop, no matter how many races he participated in. If a driver participated in 100 races, he would have more pitstop time than a driver participating in one race!
#### B. Hint: Use the sum function to accumulate the total pistop time and divide that with the amount of races the driver has participated in


In [66]:
%%sql
SELECT driver, 
AVG(milliseconds) as average
FROM pitstop_time 
GROUP BY (driver)
ORDER BY average ASC
LIMIT 20

20 rows affected.


driver,average
gasly,25044.000000000000
giovinazzi,29017.000000000000
glock,49466.555555555556
merhi,49585.230769230769
pic,52173.405405405405
karthikeyan,52388.291666666667
resta,52736.224137931034
michael_schumacher,53388.105263157895
rosa,54951.235294117647
petrov,55713.297297297297


Here we find the drivers with the least pitstop time by using the AVG function.